<a href="https://colab.research.google.com/github/Ridhtang54/python-for-finance/blob/main/Dividend_Growth_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python code for a Two-Stage Growth Model DCF valuation

# Import necessary libraries
import numpy as np

# Input parameters
initial_FCF = 100  # Initial Free Cash Flow in million dollars
g1 = 0.10  # Growth rate during high growth phase (10%)
g2 = 0.03  # Growth rate during stable phase (3%)
discount_rate = 0.08  # Discount rate (WACC or Cost of Equity) (8%)
high_growth_years = 5  # Number of years of high growth

# Function to calculate the present value of free cash flows during the high growth period
def pv_high_growth_fcf(initial_FCF, g1, discount_rate, high_growth_years):
    fcf_values = []
    for t in range(1, high_growth_years + 1):
        fcf = initial_FCF * (1 + g1) ** t
        pv_fcf = fcf / (1 + discount_rate) ** t
        fcf_values.append(pv_fcf)
    return np.sum(fcf_values)

# Function to calculate the terminal value at the end of the high growth period
def terminal_value(initial_FCF, g1, g2, discount_rate, high_growth_years):
    fcf_at_terminal = initial_FCF * (1 + g1) ** high_growth_years * (1 + g2)
    terminal_val = fcf_at_terminal / (discount_rate - g2)
    pv_terminal_value = terminal_val / (1 + discount_rate) ** high_growth_years
    return pv_terminal_value

# Calculating the present value of high growth FCFs and terminal value
pv_high_growth = pv_high_growth_fcf(initial_FCF, g1, discount_rate, high_growth_years)
pv_terminal = terminal_value(initial_FCF, g1, g2, discount_rate, high_growth_years)

# Total present value (enterprise value)
enterprise_value = pv_high_growth + pv_terminal

enterprise_value


2786.4104975905134

In [2]:
# Python code to calculate the intrinsic value of the equity share using the two-stage Dividend Discount Model

# Input parameters
D0 = 2.00  # Current dividend in Rs.
g1 = 0.20  # High growth rate (20%)
g2 = 0.10  # Stable growth rate (10%)
r = 0.15   # Required rate of return (15%)
high_growth_years = 6  # Number of years of high growth

# Step 1: Calculate the dividends during the high-growth phase
dividends = []
for t in range(1, high_growth_years + 1):
    dividend = D0 * (1 + g1) ** t
    dividends.append(dividend)

# Step 2: Calculate the terminal value at the end of year 6 (using the Gordon Growth Model)
D7 = dividends[-1] * (1 + g2)  # Dividend in year 7 (first year of stable growth)
terminal_value = D7 / (r - g2)

# Step 3: Discount the dividends and terminal value back to the present
pv_dividends = sum([dividends[t-1] / (1 + r) ** t for t in range(1, high_growth_years + 1)])
pv_terminal_value = terminal_value / (1 + r) ** high_growth_years

# Total intrinsic value
intrinsic_value = pv_dividends + pv_terminal_value
intrinsic_value


70.7649421418343

In [3]:
# Three-Stage Dividend Growth Model
def present_value(dividend, rate_of_return, time):
    return dividend / ((1 + rate_of_return) ** time)

def terminal_value(dividend, growth_rate, rate_of_return):
    return dividend / (rate_of_return - growth_rate)

def three_stage_ddm(D0, g1, g2, r, T_high, T_transition, transition_growth_rates):
    # High Growth Phase
    dividends_high = []
    for t in range(1, T_high + 1):
        Dt = D0 * (1 + g1) ** t
        dividends_high.append(Dt)

    # Transition Phase
    dividends_transition = []
    for t in range(1, T_transition + 1):
        Dt = dividends_high[-1] * (1 + transition_growth_rates[t-1])
        dividends_transition.append(Dt)

    # Terminal Value (Stable Growth Phase)
    D_last = dividends_transition[-1] * (1 + g2)
    TV = terminal_value(D_last, g2, r)

    # Discounting Dividends from High Growth Phase
    PV_high = sum([present_value(Dt, r, t) for t, Dt in enumerate(dividends_high, 1)])

    # Discounting Dividends from Transition Phase
    PV_transition = sum([present_value(Dt, r, t + T_high) for t, Dt in enumerate(dividends_transition, 1)])

    # Discounting Terminal Value
    PV_terminal = present_value(TV, r, T_high + T_transition)

    # Total Intrinsic Value
    intrinsic_value = PV_high + PV_transition + PV_terminal

    return intrinsic_value, dividends_high, dividends_transition, TV

# Parameters
D0 = 2.00  # Initial dividend in Rs
g1 = 0.20  # High growth rate (20%)
g2 = 0.10  # Stable growth rate (10%)
r = 0.15   # Required rate of return (15%)
T_high = 3  # Duration of high growth phase in years
T_transition = 3  # Duration of transition phase in years
transition_growth_rates = [0.1667, 0.1333, 0.10]  # Growth rates in the transition phase

# Calculate intrinsic value
intrinsic_value, dividends_high, dividends_transition, terminal_value = three_stage_ddm(
    D0, g1, g2, r, T_high, T_transition, transition_growth_rates)

# Output
print("Intrinsic Value of the Stock: Rs", round(intrinsic_value, 2))
print("Dividends during High Growth Phase:", [round(D, 2) for D in dividends_high])
print("Dividends during Transition Phase:", [round(D, 2) for D in dividends_transition])
print("Terminal Value at End of Transition Phase: Rs", round(terminal_value, 2))


Intrinsic Value of the Stock: Rs 48.59
Dividends during High Growth Phase: [2.4, 2.88, 3.46]
Dividends during Transition Phase: [4.03, 3.92, 3.8]
Terminal Value at End of Transition Phase: Rs 83.64
